In [1]:
# !nvidia-smi

In [2]:
import hashlib
import json
import re
import time
from typing import List

import pandas as pd
from tqdm import tqdm
import ollama
import os

## Funciones de limpieza y utilidades

In [3]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"<br\s*/?>", " ", text, flags=re.I)
    text = re.sub(r"<[^>]+>", "", text)
    return text


def chunk_by_chars(text: str, max_chars: int = 4000) -> List[str]:
    """Divide el texto en trozos por frases para no cortar a lo bruto."""
    if len(text) <= max_chars:
        return [text]
    parts, current, count = [], [], 0
    sentences = re.split(r"(?<=[\.\!\?])\s+", text)
    for s in sentences:
        if count + len(s) + 1 > max_chars and current:
            parts.append(" ".join(current).strip())
            current, count = [s], len(s) + 1
        else:
            current.append(s)
            count += len(s) + 1
    if current:
        parts.append(" ".join(current).strip())
    return parts


def hash_text(text: str) -> str:
    return hashlib.sha256(text.encode("utf-8")).hexdigest()[:16]


def save_minimal(df, out_path, summary_col):
    required = ["id", "nombre", summary_col]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise SystemExit(f"Faltan columnas para exportar {missing}. Debe haber id, nombre y {summary_col}.")
    df[["id", "nombre", summary_col]].to_csv(out_path, index=False)

# Descripcion

## Función para resumir texto con Ollama

In [4]:
def call_ollama_summary_descripcion(
    text: str,
    model: str,
    max_words: int = 80,
    temperature: float = 0.2,
    num_predict: int = 240,
    retries: int = 3,
    sleep_sec: float = 1.5,
) -> str:
    """
    Pide un resumen en formato JSON {"resumen": "..."} al modelo local de Ollama.
    """
    system_prompt = (
        "Eres un asistente que resume texto en ESPAÑOL de forma clara, fiel y concisa. "
        f"Devuelve SOLO un JSON con la clave 'resumen', con un máximo de {max_words} palabras. "
        "Mantén lo esencial (quién, qué, para qué) y nombres propios; elimina relleno, URLs y jerga. "
        "Si la descripción está vacía o irrelevante, devuelve resumen vacío."
    )

    user_prompt = f"Descripción:\n{text}\n\nDevuelve: {{\"resumen\": \"...\"}}"

    for attempt in range(1, retries + 1):
        try:
            resp = ollama.generate(
                model=model,
                prompt=f"<<SYS>>{system_prompt}<</SYS>>\n\n{user_prompt}",
                options={"temperature": temperature, "num_predict": num_predict},
                format="json",
                stream=False,
            )
            raw = resp.get("response", "").strip()
            data = json.loads(raw)
            resumen = data.get("resumen", "").strip()
            resumen = re.sub(r"\s+", " ", resumen)
            resumen_words = resumen.split()
            if len(resumen_words) > max_words:
                resumen = " ".join(resumen_words[:max_words])
            return resumen
        except Exception as e:
            if attempt == retries:
                return ""
            time.sleep(sleep_sec * attempt)
    return ""


## Parámetros de entrada

In [5]:
IN_CSV = "./data/grados.csv"      
OUT_CSV = "./data/grados_processed_descripcion.csv"           
TEXT_COL = "descripcion"                   
SUMMARY_COL = "resumen_descripcion"                    
MODEL = "llama3.1:8b"                      
MAX_WORDS = 75
SAVE_EVERY = 1
SKIP_EXISTING = True
MAX_CHARS = 4000

## Procesamiento principal

In [6]:
df = pd.read_csv(IN_CSV, sep=';')
df

,id,nombre,id_area,descripcion,salidas
0,1,Antropologia Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
1,2,Antropologia Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
2,3,Bellas Artes,2,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...
3,4,Conservacion y Restauracion del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...
4,5,Artes Escenicas,2,Este grado está adaptado a la realidad profesi...,Estos profesionales tendrán una amplia proyecc...
...,...,...,...,...,...
216,217,Ciencia de Datos e Inteligencia Artificial,75,El grado cubre la creciente necesidad de perfi...,Dichos profesionales estarán capacitados para ...
217,218,Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
218,219,Computacion e Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
219,220,Ingenieria Robotica Sotware,75,El objetivo de estas titulaciones es formar pr...,La robótica se ha utilizado desde hace años en...


In [7]:
if TEXT_COL not in df.columns:
    raise SystemExit(f"La columna '{TEXT_COL}' no existe en el CSV.")

if SUMMARY_COL not in df.columns:
    df[SUMMARY_COL] = ""

processed = 0

for i in tqdm(range(len(df)), desc="Resumiendo"):
    if SKIP_EXISTING and isinstance(df.at[i, SUMMARY_COL], str) and df.at[i, SUMMARY_COL].strip():
        continue

    raw_text = df.at[i, TEXT_COL]
    text = clean_text(raw_text)

    if not text:
        df.at[i, SUMMARY_COL] = ""
    else:
        chunks = chunk_by_chars(text, max_chars=MAX_CHARS)
        if len(chunks) == 1:
            resumen = call_ollama_summary_descripcion(chunks[0], model=MODEL, max_words=MAX_WORDS)
        else:
            partials = [
                call_ollama_summary_descripcion(ch, model=MODEL, max_words=max(40, MAX_WORDS // 2))
                for ch in chunks
            ]
            merged = " ".join([p for p in partials if p]).strip()
            resumen = call_ollama_summary_descripcion(merged, model=MODEL, max_words=MAX_WORDS)

        df.at[i, SUMMARY_COL] = resumen

    processed += 1
    if processed % SAVE_EVERY == 0:
        save_minimal(df, OUT_CSV, SUMMARY_COL)

Resumiendo: 100%|██████████| 221/221 [20:04<00:00,  5.45s/it]


In [8]:
# Guardado final
save_minimal(df, OUT_CSV, SUMMARY_COL)
print(f"✅ Listo → {OUT_CSV}")

df.head()

✅ Listo → ./data/grados_processed_descripcion.csv


,id,nombre,id_area,descripcion,salidas,resumen_descripcion
0,1,Antropologia Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...,Conoce y aprecia la diversidad de sociedades y...
1,2,Antropologia Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...,La titulación ofrece conocimiento sobre la div...
2,3,Bellas Artes,2,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...,El Grado en Bellas Artes forma artistas plásti...
3,4,Conservacion y Restauracion del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...,El Grado en Conservación y Restauración del Pa...
4,5,Artes Escenicas,2,Este grado está adaptado a la realidad profesi...,Estos profesionales tendrán una amplia proyecc...,El Grado en Artes Escénicas capacita a jóvenes...


In [9]:
cols = list(df.columns)

if "descripcion" not in df.columns or "resumen_descripcion" not in df.columns:
    raise ValueError("Faltan columnas 'descripcion' o 'resumen_descripcion' en el DataFrame.")

idx = cols.index("descripcion")
df = df.drop(columns=["descripcion"])
df = df.rename(columns={"resumen_descripcion": "descripcion"})

cols_new = [c for c in df.columns if c != "descripcion"]
cols_new.insert(idx, "descripcion")

df = df[cols_new]

df.to_csv("./data/grados_subs.csv", sep=";", index=False)

print("✅ Archivo guardado en ./data/grados_subs.csv")
df

✅ Archivo guardado en ./data/grados_subs.csv


,id,nombre,id_area,descripcion,salidas
0,1,Antropologia Social y Cultural,1,Conoce y aprecia la diversidad de sociedades y...,Los graduados en esta titulación podrán des em...
1,2,Antropologia Social y Cultural,15,La titulación ofrece conocimiento sobre la div...,Los graduados en esta titulación podrán des em...
2,3,Bellas Artes,2,El Grado en Bellas Artes forma artistas plásti...,Las profesiones que estos titulados pueden eje...
3,4,Conservacion y Restauracion del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...
4,5,Artes Escenicas,2,El Grado en Artes Escénicas capacita a jóvenes...,Estos profesionales tendrán una amplia proyecc...
...,...,...,...,...,...
216,217,Ciencia de Datos e Inteligencia Artificial,75,El grado forma profesionales versátiles con só...,Dichos profesionales estarán capacitados para ...
217,218,Inteligencia Artificial,75,Este Grado en Inteligencia Artificial proporci...,Estratega de entorno digital Especialista en c...
218,219,Computacion e Inteligencia Artificial,75,Este Grado en Informática con Inteligencia Art...,Estratega de entorno digital Especialista en c...
219,220,Ingenieria Robotica Sotware,75,"Formar profesionales que analicen, diseñen y o...",La robótica se ha utilizado desde hace años en...


# Salidas

## Función para resumir texto con Ollama

In [10]:
def call_ollama_summary_salidas(
    text: str,
    model: str,
    max_words: int = 80,
    temperature: float = 0.2,
    num_predict: int = 240,
    retries: int = 3,
    sleep_sec: float = 1.5,
) -> str:
    """
    Pide una lista en formato JSON {"resumen": "..."} al modelo local de Ollama.
    """
    system_prompt = (
        "Eres un asistente que resume texto en ESPAÑOL de forma clara, fiel y concisa. "
        f"Devuelve SOLO un JSON con la clave 'resumen', con un máximo de {max_words} palabras. "
        "El resumen tiene que estar compuesto por elementos únicos de varias palabras separados por -. "
        "Mantén lo esencial sobre la información de las salidas y nombres propios; elimina relleno, URLs y jerga. "
        "Si las salidas no continenen informacion o son irrelevantes, devuelve resumen vacío."
    )

    user_prompt = f"Salidas:\n{text}\n\nDevuelve: {{\"resumen\": \"...\"}}"

    for attempt in range(1, retries + 1):
        try:
            resp = ollama.generate(
                model=model,
                prompt=f"<<SYS>>{system_prompt}<</SYS>>\n\n{user_prompt}",
                options={"temperature": temperature, "num_predict": num_predict},
                format="json",
                stream=False,
            )
            raw = resp.get("response", "").strip()
            data = json.loads(raw)
            resumen = data.get("resumen", "").strip()
            resumen = re.sub(r"\s+", " ", resumen)
            resumen_words = resumen.split()
            if len(resumen_words) > max_words:
                resumen = " ".join(resumen_words[:max_words])
            return resumen
        except Exception as e:
            if attempt == retries:
                return ""
            time.sleep(sleep_sec * attempt)
    return ""


## Parámetros de entrada

In [11]:
IN_CSV = "./data/grados_subs.csv"      
OUT_CSV = "./data/grados_processed_salidas.csv"           
TEXT_COL = "salidas"                   
SUMMARY_COL = "resumen_salidas"                    
MODEL = "llama3.1:8b"                      
MAX_WORDS = 75
SAVE_EVERY = 25
SKIP_EXISTING = True
MAX_CHARS = 4000

## Procesamiento principal

In [12]:
df = pd.read_csv(IN_CSV, sep=";")
if TEXT_COL not in df.columns:
    raise SystemExit(f"La columna '{TEXT_COL}' no existe en el CSV.")

if SUMMARY_COL not in df.columns:
    df[SUMMARY_COL] = ""

processed = 0

for i in tqdm(range(len(df)), desc="Resumiendo"):
    if SKIP_EXISTING and isinstance(df.at[i, SUMMARY_COL], str) and df.at[i, SUMMARY_COL].strip():
        continue

    raw_text = df.at[i, TEXT_COL]
    text = clean_text(raw_text)

    if not text:
        df.at[i, SUMMARY_COL] = ""
    else:
        chunks = chunk_by_chars(text, max_chars=MAX_CHARS)
        if len(chunks) == 1:
            resumen = call_ollama_summary_salidas(chunks[0], model=MODEL, max_words=MAX_WORDS)
        else:
            partials = [
                call_ollama_summary_salidas(ch, model=MODEL, max_words=max(40, MAX_WORDS // 2))
                for ch in chunks
            ]
            merged = " ".join([p for p in partials if p]).strip()
            resumen = call_ollama_summary_salidas(merged, model=MODEL, max_words=MAX_WORDS)

        df.at[i, SUMMARY_COL] = resumen

    processed += 1
    if processed % SAVE_EVERY == 0:
        save_minimal(df, OUT_CSV, SUMMARY_COL)

Resumiendo: 100%|██████████| 221/221 [16:43<00:00,  4.54s/it]


In [13]:
# Guardado final
save_minimal(df, OUT_CSV, SUMMARY_COL)
print(f"✅ Listo → {OUT_CSV}")

df.head()

✅ Listo → ./data/grados_processed_salidas.csv


,id,nombre,id_area,descripcion,salidas,resumen_salidas
0,1,Antropologia Social y Cultural,1,Conoce y aprecia la diversidad de sociedades y...,Los graduados en esta titulación podrán des em...,"Graduados en diversidad cultural, patrimonio e..."
1,2,Antropologia Social y Cultural,15,La titulación ofrece conocimiento sobre la div...,Los graduados en esta titulación podrán des em...,"Graduados en diversidad cultural, patrimonio e..."
2,3,Bellas Artes,2,El Grado en Bellas Artes forma artistas plásti...,Las profesiones que estos titulados pueden eje...,"Pintor, escultor, dibujante, fotógrafo, crític..."
3,4,Conservacion y Restauracion del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...,ConservadorRestaurador en instituciones públic...
4,5,Artes Escenicas,2,El Grado en Artes Escénicas capacita a jóvenes...,Estos profesionales tendrán una amplia proyecc...,Profesionales con proyección multidisciplinar ...


In [14]:
cols = list(df.columns)

if "salidas" not in df.columns or "resumen_salidas" not in df.columns:
    raise ValueError("Faltan columnas 'salidas' o 'resumen_salidas' en el DataFrame.")

idx = cols.index("salidas")
df = df.drop(columns=["salidas"])
df = df.rename(columns={"resumen_salidas": "salidas"})

cols_new = [c for c in df.columns if c != "salidas"]
cols_new.insert(idx, "salidas")

df = df[cols_new]

df.to_csv("./data/grados_subs.csv", sep=";", index=False)

print("✅ Archivo guardado en ./data/grado_subs.csv")
df

✅ Archivo guardado en ./data/grado_subs.csv


,id,nombre,id_area,descripcion,salidas
0,1,Antropologia Social y Cultural,1,Conoce y aprecia la diversidad de sociedades y...,"Graduados en diversidad cultural, patrimonio e..."
1,2,Antropologia Social y Cultural,15,La titulación ofrece conocimiento sobre la div...,"Graduados en diversidad cultural, patrimonio e..."
2,3,Bellas Artes,2,El Grado en Bellas Artes forma artistas plásti...,"Pintor, escultor, dibujante, fotógrafo, crític..."
3,4,Conservacion y Restauracion del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,ConservadorRestaurador en instituciones públic...
4,5,Artes Escenicas,2,El Grado en Artes Escénicas capacita a jóvenes...,Profesionales con proyección multidisciplinar ...
...,...,...,...,...,...
216,217,Ciencia de Datos e Inteligencia Artificial,75,El grado forma profesionales versátiles con só...,"Capacitados en Big Data, Inteligencia Artifici..."
217,218,Inteligencia Artificial,75,Este Grado en Inteligencia Artificial proporci...,"Especialistas en tecnología digital, incluyend..."
218,219,Computacion e Inteligencia Artificial,75,Este Grado en Informática con Inteligencia Art...,"Especialistas en tecnología digital, incluyend..."
219,220,Ingenieria Robotica Sotware,75,"Formar profesionales que analicen, diseñen y o...",La robótica se utiliza en industrias innovador...
